In [27]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler,RobustScaler
import csv
import statsmodels.api as sm
from sklearn.feature_selection import SelectKBest,f_regression
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from keras.layers import Dense, Dropout, LeakyReLU

In [28]:
import os
print(os.getcwd())

d:\IML ASSIGNMENT 2\CODE


In [29]:
df = pd.read_csv('train.csv')

In [30]:
X = df.loc[:, df.columns != 'price_doc']
y = df[['price_doc']]

In [31]:
object_cols = X.select_dtypes(include=['object']).columns

In [32]:
label = LabelEncoder()
for col in object_cols:
    X[col] = label.fit_transform(X[col].astype(str))

C:\Users\Irtiza\AppData\Local\Temp\ipykernel_15748\2966562952.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = label.fit_transform(X[col].astype(str))
C:\Users\Irtiza\AppData\Local\Temp\ipykernel_15748\2966562952.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = label.fit_transform(X[col].astype(str))
C:\Users\Irtiza\AppData\Local\Temp\ipykernel_15748\2966562952.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [33]:
X.shape

(181507, 271)

In [34]:
num_cols = X.select_dtypes(include=['int64','float64']).columns

In [35]:
from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
scaler = RobustScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])

C:\Users\Irtiza\AppData\Local\Temp\ipykernel_15748\3760893776.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[num_cols] = scaler.fit_transform(X[num_cols])


In [36]:
X.shape

(181507, 271)

In [37]:
from sklearn.decomposition import PCA
pca = PCA(n_components=15)
X = pca.fit_transform(X)

In [38]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=3)
X = poly.fit_transform(X)

In [39]:
X.shape

(181507, 816)

In [40]:
df_test = pd.read_csv('test.csv')

In [41]:
df_test = df_test.drop(['row ID'], axis=1)

In [42]:
label_test = LabelEncoder()
for col in object_cols:
    df_test[col] = label_test.fit_transform(df_test[col].astype(str))


In [43]:
df_test[num_cols] = scaler.transform(df_test[num_cols])

In [44]:
df_test.shape

(77789, 271)

In [45]:
df_test = pca.transform(df_test)

In [46]:
df_test = poly.transform(df_test)

In [47]:
df_test.shape

(77789, 816)

In [48]:
#convert all int64 to float32
X = X.astype('float32')
df_test = df_test.astype('float32')

In [49]:
import torch
os.environ["KERAS_BACKEND"] = "torch"
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l1, l2
from keras.optimizers import RMSprop,Adam,Adamax
from keras.callbacks import EarlyStopping

In [50]:
# Model
n_features = X.shape[1]
model = Sequential()
model.add(Dense(100, input_dim=n_features))
model.add(LeakyReLU(alpha=0.01))
model.add(Dropout(0.2))
model.add(Dense(50))
model.add(LeakyReLU(alpha=0.01))
model.add(Dropout(0.2))
model.add(Dense(25))
model.add(LeakyReLU(alpha=0.01))
model.add(Dropout(0.2))
model.add(Dense(1, activation='linear'))
model.compile(optimizer=Adamax(learning_rate=0.01), loss='mean_squared_error')

# Print the model summary
print(model.summary())


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 100)               81700     
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 100)               0         
                                                                 


 dropout_3 (Dropout)         (None, 100)               0         
                                                                 
 dense_5 (Dense)             (None, 50)                5050      
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 50)                0         
                                                                 
 dropout_4 (Dropout)         (None, 50)                0         
                                                                 
 dense_6 (Dense)             (None, 25)                1275      
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 25)                0         
                                                                 
 dropout_5 (Dropout)         (None, 25)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 26        
          

In [51]:
# Model training
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
model.fit(X, y, epochs=100, batch_size=256, callbacks=[early_stopping])

y_pred = model.predict(df_test)


Epoch 1/100
710/710 [==============================] - 5s 5ms/step - loss: 303371424628736.0000
Epoch 2/100
710/710 [==============================] - 4s 5ms/step - loss: 254299460337664.0000
Epoch 3/100
710/710 [==============================] - 4s 5ms/step - loss: 240470118629376.0000
Epoch 4/100
710/710 [==============================] - 4s 5ms/step - loss: 234729173417984.0000
Epoch 5/100
710/710 [==============================] - 4s 6ms/step - loss: 229247218089984.0000
Epoch 6/100
710/710 [==============================] - 4s 5ms/step - loss: 227247038398464.0000
Epoch 7/100
710/710 [==============================] - 4s 5ms/step - loss: 225443705782272.0000
Epoch 8/100
710/710 [==============================] - 4s 5ms/step - loss: 220853694365696.0000
Epoch 9/100
710/710 [==============================] - 4s 5ms/step - loss: 220324473864192.0000
Epoch 10/100
710/710 [==============================] - 4s 5ms/step - loss: 219292624748544.0000
Epoch 11/100
710/710 [=================

In [52]:
filepath = 'neural_29nov_3.csv'
with open(filepath, mode='w', newline='') as file: 
    writer = csv.writer(file)
    c=1
    writer.writerow(['row ID', 'price_doc'])
    for i in y_pred:
        writer.writerow([c ,i[0]])
        c=c+1